# Import

In [1]:
import numpy as np
import pandas as pd
import os
import glob
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import *
from sklearn.metrics import *
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [24]:
# GOOD_FEATURES = [1,3,4,5,6,8,10,11,12,17,18,19,20,
#                  21,22,23,24,26,28,30,31,34,
#                  35,36,37,38,41,42,44,45,48,50,
#                  53,55,57,58,60,61,62,64,65,67,68,
#                  69,70,72,75]

In [25]:
GOOD_FEATURES = [1,3,5,11,17,19,21,23,31,35,37,41,48,50,58,60,62,64,68,70,72,75]

In [26]:
TRAIN_FEATURES = []
for i in range(52):
    [TRAIN_FEATURES.append(x) for x in train.columns[2 + 76*i : 2 + 76*(i+1)]]

# TRAIN_FEATURES = [x for x in TRAIN_FEATURES if 'mean' in x]
# TRAIN_FEATURES = [x for x in TRAIN_FEATURES if '_'.join(x.split('_')[:-1]) in FIP[:10]]

# TRAIN_FEATURES = [x for x in TRAIN_FEATURES if 'mean' in x]
TRAIN_FEATURES = [x for x in TRAIN_FEATURES if int(x.split('_')[-1]) in GOOD_FEATURES]

NUMF = int(len(TRAIN_FEATURES) / len(GOOD_FEATURES))
    
GEN_FEATURES = []
for i in range(NUMF):
    GEN_FEATURES.append(TRAIN_FEATURES[i*len(GOOD_FEATURES):(i+1)*len(GOOD_FEATURES)])
    
GEN_FEATURES_NAMES = []
for i in range(NUMF):
    GEN_FEATURES_NAMES.append('_'.join(TRAIN_FEATURES[i*len(GOOD_FEATURES):(i+1)*len(GOOD_FEATURES)][0].split('_')[:-1]))

In [27]:
# for g, n in zip(GEN_FEATURES, GEN_FEATURES_NAMES):
#     plt.plot(train[g].iloc[0].dropna())
#     plt.title(n)
#     plt.show()

# generate train

In [28]:
len(GOOD_FEATURES) / 6

3.6666666666666665

In [30]:
train_aggs = []

for i in tqdm(range(len(GEN_FEATURES_NAMES))):
    
    train_agg = pd.DataFrame()
    
    FEATURE_NAME = GEN_FEATURES_NAMES[i]
    FEATURES = GEN_FEATURES[i]
    
    ndvi_train = train[FEATURES].copy()
    
    train_agg = ndvi_train.copy()
    
    for i in range(1, len(FEATURES)):
        train_agg['speed_{}'.format(FEATURES[i])] = train_agg[FEATURES[i]] - train_agg[FEATURES[i-1]]
    
#     for i in range(0, 76, 3):
#         train_agg['mean_{}_{}'.format(FEATURE_NAME, i)] = ndvi_train[ndvi_train.columns[i:i+3]].mean(axis=1)
#     for i in range(0, 76, 6):
#         train_agg['mean_{}_{}'.format(FEATURE_NAME, i)] = ndvi_train[ndvi_train.columns[i:i+6]].mean(axis=1)
            
    train_agg['{}_std_'.format(FEATURE_NAME)] = ndvi_train.std(axis=1) 
    train_agg['{}_mean_'.format(FEATURE_NAME)] = ndvi_train.mean(axis=1) 
    train_agg['{}_min_'.format(FEATURE_NAME)] = ndvi_train.min(axis=1) 
    train_agg['{}_max_'.format(FEATURE_NAME)] = ndvi_train.max(axis=1)
    
#     print('Base done!')

#     for i in range(1, 10, 2): 
#         train_agg['{}_quant_{}'.format(FEATURE_NAME, i)] = ndvi_train.quantile(i/10, axis=1)
        
#     print('Quantile done!')
    
    def max_day(x):
        try:
            return np.where(x == x.max())[0][0]
        except:
            return np.nan
    def min_day(x):
        try:
            return np.where(x == x.min())[0][0]
        except:
            return np.nan
    
    maxd = ndvi_train.apply(lambda x: max_day(x), axis=1)
    mindl = ndvi_train.apply(lambda x: min_day(x[:11]), axis=1)
    mindr = ndvi_train.apply(lambda x: 11+min_day(x[11:]), axis=1)
    
    train_agg['{}_max_day'.format(FEATURE_NAME)] = maxd
    train_agg['{}_min_day_left'.format(FEATURE_NAME)] = mindl
    train_agg['{}_min_day_right'.format(FEATURE_NAME)] = mindr
    
#     print('Days done!')
    
    train_agg['{}_dur_left'.format(FEATURE_NAME)] = np.abs(maxd - mindl)
    train_agg['{}_dur_right'.format(FEATURE_NAME)] = np.abs(maxd - mindr)
    
#     print('Dur done!')
        
    for i in range(1, 10, 2):
        
        def day_more_quant(x):
            return len(np.where(x.values[:76] > np.nanquantile(x.values[:76], i/10))[0])
        def mean_day_more_quant(x):
            return np.mean(np.where(x.values[:76] > np.nanquantile(x.values[:76], i/10))[0])
        
        train_agg['{}_more_quant_{}'.format(FEATURE_NAME, i)] = ndvi_train.apply(lambda x: day_more_quant(x), axis=1)
        train_agg['{}_mean_more_quant_{}'.format(FEATURE_NAME, i)] = ndvi_train.apply(lambda x: mean_day_more_quant(x), axis=1)

    train_aggs.append(train_agg)

100%|█████████████████████████████████████████| 52/52 [1:09:07<00:00, 79.77s/it]


In [31]:
len(train_aggs) * train_agg.shape[1]

3224

In [32]:
train_aggs_df = pd.concat(train_aggs, axis=1)

In [33]:
len(set(train_aggs_df.columns))

3224

In [48]:
train_aggs_df['field_id'] = train['field_id']

/var/folders/8n/c9njytgn0g778_3wm9c937c80000gn/T/ipykernel_46459/4178270243.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  train_aggs_df['field_id'] = train['field_id']


In [50]:
train_aggs_df.to_feather('train_agg_2.ftr')

In [53]:
train_aggs_df.to_csv('new_data/train_agg_2.csv')

In [52]:
train_aggs_df

,ndvi_mean_1,ndvi_mean_3,ndvi_mean_5,ndvi_mean_11,ndvi_mean_17,ndvi_mean_19,ndvi_mean_21,ndvi_mean_23,ndvi_mean_31,ndvi_mean_35,...,B12_median_mean_more_quant_1,B12_median_more_quant_3,B12_median_mean_more_quant_3,B12_median_more_quant_5,B12_median_mean_more_quant_5,B12_median_more_quant_7,B12_median_mean_more_quant_7,B12_median_more_quant_9,B12_median_mean_more_quant_9,field_id
0,0.462953,0.459135,0.413491,0.340328,0.354241,0.353121,0.351588,0.433098,0.422065,0.424641,...,11.000000,15,13.400000,10,16.100000,7,17.857143,3,20.000000,877
1,0.217767,0.228084,0.202413,0.210728,0.346269,0.376585,0.372421,0.450842,0.430563,0.433313,...,11.000000,15,11.133333,11,11.909091,7,9.428571,3,1.666667,4121
2,0.141454,0.157054,0.132902,0.148366,0.256331,0.292342,0.300801,0.360497,0.356821,0.349519,...,11.000000,13,11.538462,11,11.909091,7,9.428571,3,1.000000,7585
3,0.479385,0.471451,0.419420,0.337496,0.327056,0.345388,0.360373,0.419500,0.424403,0.412920,...,11.000000,13,14.769231,11,15.818182,7,17.857143,3,20.000000,7803
4,0.298645,0.293928,0.279787,0.235267,0.231145,0.220772,0.208921,0.288813,0.289832,0.331418,...,11.000000,15,11.333333,11,14.181818,7,15.571429,3,12.666667,15629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87107,0.301762,0.330588,0.317131,0.308447,0.418673,0.475420,0.502678,0.618766,0.739399,0.752555,...,10.789474,14,10.642857,11,10.454545,6,6.000000,2,11.000000,118054
87108,0.197949,0.226795,0.207473,0.206080,0.365618,0.452137,0.471816,0.547988,0.570046,0.564271,...,10.894737,14,10.857143,11,11.636364,7,11.000000,3,17.000000,119307
87109,0.275856,0.413481,0.437516,0.292054,0.397613,0.467396,0.487946,0.606768,0.543147,0.494141,...,10.222222,15,10.600000,11,9.272727,7,8.142857,2,12.000000,120055
87110,0.173944,0.157304,0.137431,0.141951,0.354560,0.565079,0.625546,0.756699,0.795109,0.749857,...,10.944444,14,10.428571,11,10.090909,7,9.428571,2,10.500000,120670


# generate test

In [38]:
len(GOOD_FEATURES) / 6

3.6666666666666665

In [39]:
test_aggs = []

for i in tqdm(range(len(GEN_FEATURES_NAMES))):
    
    train_agg = pd.DataFrame()
    
    FEATURE_NAME = GEN_FEATURES_NAMES[i]
    FEATURES = GEN_FEATURES[i]
    
    ndvi_train = test[FEATURES].copy()
    
    train_agg = ndvi_train.copy()
    
    for i in range(1, len(FEATURES)):
        train_agg['speed_{}'.format(FEATURES[i])] = train_agg[FEATURES[i]] - train_agg[FEATURES[i-1]]
    
#     for i in range(0, 76, 3):
#         train_agg['mean_{}_{}'.format(FEATURE_NAME, i)] = ndvi_train[ndvi_train.columns[i:i+3]].mean(axis=1)
#     for i in range(0, 76, 6):
#         train_agg['mean_{}_{}'.format(FEATURE_NAME, i)] = ndvi_train[ndvi_train.columns[i:i+6]].mean(axis=1)
            
    train_agg['{}_std_'.format(FEATURE_NAME)] = ndvi_train.std(axis=1) 
    train_agg['{}_mean_'.format(FEATURE_NAME)] = ndvi_train.mean(axis=1) 
    train_agg['{}_min_'.format(FEATURE_NAME)] = ndvi_train.min(axis=1) 
    train_agg['{}_max_'.format(FEATURE_NAME)] = ndvi_train.max(axis=1)
    
#     print('Base done!')

#     for i in range(1, 10, 2): 
#         train_agg['{}_quant_{}'.format(FEATURE_NAME, i)] = ndvi_train.quantile(i/10, axis=1)
        
#     print('Quantile done!')
    
    def max_day(x):
        try:
            return np.where(x == x.max())[0][0]
        except:
            return np.nan
    def min_day(x):
        try:
            return np.where(x == x.min())[0][0]
        except:
            return np.nan
    
    maxd = ndvi_train.apply(lambda x: max_day(x), axis=1)
    mindl = ndvi_train.apply(lambda x: min_day(x[:11]), axis=1)
    mindr = ndvi_train.apply(lambda x: 11+min_day(x[11:]), axis=1)
    
    train_agg['{}_max_day'.format(FEATURE_NAME)] = maxd
    train_agg['{}_min_day_left'.format(FEATURE_NAME)] = mindl
    train_agg['{}_min_day_right'.format(FEATURE_NAME)] = mindr
    
#     print('Days done!')
    
    train_agg['{}_dur_left'.format(FEATURE_NAME)] = np.abs(maxd - mindl)
    train_agg['{}_dur_right'.format(FEATURE_NAME)] = np.abs(maxd - mindr)
    
#     print('Dur done!')
        
    for i in range(1, 10, 2):
        
        def day_more_quant(x):
            return len(np.where(x.values[:76] > np.nanquantile(x.values[:76], i/10))[0])
        def mean_day_more_quant(x):
            return np.mean(np.where(x.values[:76] > np.nanquantile(x.values[:76], i/10))[0])
        
        train_agg['{}_more_quant_{}'.format(FEATURE_NAME, i)] = ndvi_train.apply(lambda x: day_more_quant(x), axis=1)
        train_agg['{}_mean_more_quant_{}'.format(FEATURE_NAME, i)] = ndvi_train.apply(lambda x: mean_day_more_quant(x), axis=1)

    test_aggs.append(train_agg)
    
    

  0%|                                                    | 0/52 [00:00<?, ?it/s]/Users/hare/opt/anaconda3/envs/champ/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1374: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(
/Users/hare/opt/anaconda3/envs/champ/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/hare/opt/anaconda3/envs/champ/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|███████████████████████████████████████████| 52/52 [27:46<00:00, 32.05s/it]


In [40]:
len(test_aggs) * train_agg.shape[1]

3224

In [41]:
test_aggs_df = pd.concat(test_aggs, axis=1)

In [42]:
len(set(train_aggs_df.columns))

3224

In [45]:
test_aggs_df['field_id'] = test['field_id']

/var/folders/8n/c9njytgn0g778_3wm9c937c80000gn/T/ipykernel_46459/3618799839.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  test_aggs_df['field_id'] = test['field_id']


In [46]:
test_aggs_df

,ndvi_mean_1,ndvi_mean_3,ndvi_mean_5,ndvi_mean_11,ndvi_mean_17,ndvi_mean_19,ndvi_mean_21,ndvi_mean_23,ndvi_mean_31,ndvi_mean_35,...,B12_median_mean_more_quant_1,B12_median_more_quant_3,B12_median_mean_more_quant_3,B12_median_more_quant_5,B12_median_mean_more_quant_5,B12_median_more_quant_7,B12_median_mean_more_quant_7,B12_median_more_quant_9,B12_median_mean_more_quant_9,field_id
0,0.154170,0.142463,0.132454,0.140609,0.190295,0.015616,0.248982,0.306265,0.345342,0.314735,...,10.947368,15,11.800000,11,12.000000,7,13.714286,3,13.333333,6617
1,0.134280,0.138282,0.130676,0.132036,0.219290,0.001574,0.326026,0.401208,0.459930,0.407312,...,11.210526,15,12.066667,11,13.272727,6,18.500000,2,20.000000,9213
2,0.141532,0.133670,0.120994,0.120947,0.194519,0.012191,0.270799,0.358529,0.420243,0.253764,...,11.000000,15,12.133333,11,11.545455,6,17.666667,3,19.000000,10802
3,0.158129,0.155410,0.153275,0.156023,0.208189,0.018421,0.253554,0.310938,0.344678,0.298318,...,11.000000,15,11.800000,11,12.000000,7,15.857143,3,20.000000,11307
4,0.124440,0.158890,0.149101,0.157985,0.212079,0.013632,0.245018,0.296493,0.295418,0.074106,...,11.315789,15,12.000000,11,14.727273,7,14.142857,2,9.500000,20336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35290,0.169557,0.143137,0.141836,0.138938,0.230062,0.423866,0.574363,0.687142,0.827021,0.705024,...,10.842105,15,11.600000,11,11.909091,7,9.428571,3,2.000000,104674
35291,0.146466,0.151965,0.154919,0.165772,0.246493,0.312741,0.311190,0.389059,0.384920,0.387399,...,10.684211,15,11.800000,10,11.200000,7,11.000000,3,2.000000,110622
35292,0.161462,0.132539,0.129478,0.122347,0.156169,0.252970,0.340801,0.474222,0.794218,0.794590,...,10.736842,14,11.142857,11,11.181818,7,8.285714,1,3.000000,117104
35293,0.163215,0.169548,0.160455,0.170806,0.242693,0.295759,0.311597,0.375198,0.405368,0.425575,...,10.684211,15,11.800000,11,15.181818,6,16.833333,1,21.000000,117436


In [47]:
test_aggs_df.to_feather('test_agg_2.ftr')

In [54]:
test_aggs_df.to_csv('new_data/test_agg_2.csv')